# Create Cooperative
This notebook contains snippets to create COOP from scratch given a NR.  It also contains functionality to update good standing status for a business and also to reset/remove voluntary dissolution data after a manual voluntary dissolution filing.

## Setup environment
Before running following environment setup snippets, ensure environment variables found in `default-bcr-business-setup-TEST` notebook contain the correct values.  If there is a need to run update the good standing state of a business or to reset/remove voluntary dissolution data for a given business, ensure you have port forwarded the lear database and the corresponding database config in `default-bcr-business-setup-TEST` is up to date.

In [ ]:
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/legal_api_utils.ipynb
%run /workspaces/lear/tests/data/common/legal_api_utils_test_payload_data.ipynb
%run /workspaces/lear/tests/data/common/legal_api_utils_test_data.ipynb
%run /workspaces/lear/tests/data/common/namex_api_utils.ipynb
%run /workspaces/lear/tests/data/common/auth_api_utils.ipynb

In [ ]:
import copy
import re
import json
from legal_api.core import Filing as CoreFiling
from legal_api.models import Business, Comment, Filing, RegistrationBootstrap
from legal_api.utils.datetime import date, datetime

In [ ]:
# account to associate business to e.g. 2596(DEV BCREG0018 AC Dev Account), 1009(TEST BCREG0029, AC Test Account)
# account id can be found by selecting account of interest in My Business Registry site and referencing url.
# e.g. https://dev.bcregistry.ca/business/auth/account/2596/business
account = 2596
# Provide own identifier if not creating business from scratch and want to update good standing or reset voluntary dissolution
# data for a given business.
# identifier = 'CP1XXXXXX'
legal_type = 'CP'
nr_num = '<nr_num_placeholder>' # newly unaffiliated and unconsumed nr num.  expected format "NR 6771356"
email = '<placeholder_email_address>'
verify_ssl=True # set to False if using proxy to debug requests
account, legal_type, nr_num, email

### Get NR Info

In [ ]:
approved_nr_name = get_nr_approved_name(nr_num, email, verify_ssl)
assert approved_nr_name
approved_nr_name

## Tokens

In [ ]:
auth_token = get_auth_token(verify_ssl)
assert auth_token
# auth_token

In [ ]:
lear_token = get_lear_token(verify_ssl)
assert lear_token
# lear_token

## Affiliate NR to account

In [ ]:
r = affiliate_nr_to_account(auth_token, nr_num, email, account, verify_ssl)
r.status_code, r.text

### Affiliate NR to entity

In [ ]:
r = affiliate_nr_to_entity(auth_token, nr_num, approved_nr_name, verify_ssl)
r.status_code, r.text

## Create Draft IA

In [ ]:
payload = factory_empty_ia_payload()
payload['filing']['header']['accountId'] = account
payload['filing']['business']['legalType'] = legal_type
payload['filing']['incorporationApplication']['nameRequest']['legalType'] = legal_type
payload['filing']['incorporationApplication']['nameRequest']['nrNumber'] = nr_num

r = create_empty_draft_ia(lear_token, payload, verify_ssl)
response_dict = json.loads(r.text)
temp_identifier = response_dict['filing']['business']['identifier']
filing_id = response_dict['filing']['header']['filingId']
assert r.status_code == 201
r.status_code, temp_identifier, filing_id, r.text


In [ ]:
# upload rules file to minio
rules_file_name = 'rules_file_name.pdf'
rules_file_key, rules_file_presigned_url = factory_file_upload(lear_token, rules_file_name, verify_ssl)
assert rules_file_key
assert rules_file_presigned_url
rules_file_name, rules_file_key

In [ ]:
# upload memorandum file to minio
memorandum_file_name = 'memorandum_file_name.pdf'
memorandum_file_key, memorandum_file_presigned_url = factory_file_upload(lear_token, memorandum_file_name, verify_ssl)
assert memorandum_file_key
assert memorandum_file_presigned_url
memorandum_file_name, memorandum_file_key

In [ ]:
# *****************************************************************************
# NOTE: might need to wait a few minutes after DRAFT filing is initially saved
# *****************************************************************************

payload = factory_coop_ia_payload()
payload['filing']['business']['legalType'] = legal_type
payload['filing']['business']['identifier'] = temp_identifier
payload['filing']['incorporationApplication']['nameRequest']['legalType'] = legal_type
payload['filing']['incorporationApplication']['nameRequest']['nrNumber'] = nr_num
payload['filing']['incorporationApplication']['nameRequest']['legalName'] = approved_nr_name
payload['filing']['incorporationApplication']['contactPoint']['email'] = email
payload['filing']['incorporationApplication']['parties'][0]['officer']['email'] = email
payload['filing']['incorporationApplication']['cooperative']['cooperativeAssociationType'] = legal_type
payload['filing']['incorporationApplication']['cooperative']['rulesFileKey'] = rules_file_key
payload['filing']['incorporationApplication']['cooperative']['rulesFileName'] = rules_file_name
payload['filing']['incorporationApplication']['cooperative']['memorandumFileKey'] = memorandum_file_key
payload['filing']['incorporationApplication']['cooperative']['memorandumFileName'] = memorandum_file_name

r = submit_ia(lear_token, payload, temp_identifier, filing_id, verify_ssl)
assert r.status_code == 202
response_dict = json.loads(r.text)
payment_token = response_dict['filing']['header']['paymentToken']
print(f'payment_token: {payment_token}')
r.status_code, r.text

## Create/update transaction
Note: Making a payment is not required if the `entity-notebook-service-account` service client is used for `LEGAL_SVC_CLIENT_ID`.  The payment will be automatically created/processed and the filing will be processed as well.  Otherwise, uncomment and use the payment url provided after creating the payment tranaction in the snippet below.

In [ ]:
# r = create_filing_payment_transaction(temp_identifier, payment_token, verify_ssl)
# r.status_code, r.text
# response_dict = json.loads(r.text)
# transaction_id = response_dict.get('id')
# pay_sys_url = response_dict.get('paySystemUrl')
# print('Use following link to make payment in browser.  Continue with snippets below after making payment.')
# print(f'pay_sys_url: {pay_sys_url}')

## Get business identifier

In [ ]:
# Note: might need to wait a few minutes for this to propogate to auth
identifier = get_identifier_from_affiliations(auth_token, account, approved_nr_name, verify_ssl)
assert identifier
identifier

### Update good standing
Note: when creating a COOP from scratch using above steps, the COOP will be in good standing already.  Only use this snippet if there is a need to set the COOP to not in good standing or to toggle the good standing state for an existing business.

In [ ]:
assert identifier
in_good_standing = False

update_in_good_standing(identifier, in_good_standing)
print(f'updated to in good standing = {in_good_standing}')

db.session.commit()

## Data Reset
Remove voluntary dissolution filing and update/remove operational data introduced by voluntary dissolution

In [ ]:
assert identifier
reset_vd_data(identifier)
db.session.commit()
print(f'business reset - voluntary dissolution data removed')

In [ ]:
# rollback if reset didn't work properly
db.session.rollback()